In [ ]:
import os

In [ ]:
mind_data_path = "/kaggle/input/mind-data/mind_data"  
os.makedirs(mind_data_path, exist_ok=True) 

In [ ]:
!pip install tensorflow==2.15.0 "numpy<2.0"

In [ ]:
# Cài cuDNN 8.9 từ pip
!pip uninstall tensorflow -y
!pip install nvidia-cudnn-cu12==8.9.7.29
!pip install tensorflow==2.15.0

# Thiết lập LD_LIBRARY_PATH
import os
cudnn_path = '/usr/local/lib/python3.11/dist-packages/nvidia/cudnn/lib'
os.environ['LD_LIBRARY_PATH'] = f"{cudnn_path}:{os.environ.get('LD_LIBRARY_PATH', '')}"

print("✅ cuDNN 8.9 installed. Please RESTART KERNEL!")

In [ ]:
import os
cudnn_path = '/usr/local/lib/python3.11/dist-packages/nvidia/cudnn/lib'
os.environ['LD_LIBRARY_PATH'] = f"{cudnn_path}:{os.environ.get('LD_LIBRARY_PATH', '')}"

import tensorflow as tf
print("GPUs:", tf.config.list_physical_devices('GPU'))

In [ ]:
# Cell 1: Monkey patch để force embedding lên GPU
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

import tensorflow as tf

# Monkey patch Embedding layer để force GPU placement
original_embedding_init = tf.keras.layers.Embedding.__init__

def patched_embedding_init(self, *args, **kwargs):
    # Force embeddings on GPU
    with tf.device('/GPU:0'):
        original_embedding_init(self, *args, **kwargs)

tf.keras.layers.Embedding.__init__ = patched_embedding_init

print("✅ Patched Embedding layer to use GPU")

In [ ]:
# Cell 2: Setup
from recommenders.models.newsrec.newsrec_utils import prepare_hparams  
from recommenders.models.newsrec.models.nrms import NRMSModel  
from recommenders.models.newsrec.io.mind_iterator import MINDIterator  

train_news_file = os.path.join(mind_data_path, "train", "news.tsv")  
train_behaviors_file = os.path.join(mind_data_path, "train", "behaviors.tsv")  
valid_news_file = os.path.join(mind_data_path, "valid", "news.tsv")  
valid_behaviors_file = os.path.join(mind_data_path, "valid", "behaviors.tsv")  
wordEmb_file = os.path.join(mind_data_path, "utils", "embedding.npy")  
userDict_file = os.path.join(mind_data_path, "utils", "uid2index.pkl")  
wordDict_file = os.path.join(mind_data_path, "utils", "word_dict.pkl")  
yaml_file = os.path.join(mind_data_path, "utils", "nrms.yaml")  

hparams = prepare_hparams(  
    yaml_file,  
    wordEmb_file=wordEmb_file,  
    wordDict_file=wordDict_file,  
    userDict_file=userDict_file,  
    epochs=3,
    batch_size=384
)  

# Create model with GPU
with tf.device('/GPU:0'):
    iterator = MINDIterator  
    model = NRMSModel(hparams, iterator, seed=42)

print("✅ Model created on GPU")

In [ ]:
# Cell 3: Load checkpoint
checkpoint_path = "/kaggle/input/model-epoch-6-10/model/nrms_ckpt"
model.model.load_weights(checkpoint_path)
print(f"✅ Checkpoint loaded")

In [ ]:
# Cell 4: Training
print("🔥 Starting training...")
model.fit(  
    train_news_file,
    train_behaviors_file,
    valid_news_file,
    valid_behaviors_file
)
print("✅ Training completed!")

In [ ]:
# Đánh giá trên validation set  
eval_results = model.run_eval(valid_news_file, valid_behaviors_file)  
print("Kết quả đánh giá:")  
for metric, value in eval_results.items():  
    print(f"{metric}: {value:.4f}")

In [ ]:
model_path = os.path.join("/kaggle/working/", "model")
os.makedirs(model_path, exist_ok=True)

model.model.save_weights(os.path.join(model_path, "nrms_ckpt"))

In [ ]:
from tqdm import tqdm
import numpy as np

test_behaviors_file = "/kaggle/input/mind-data/mind_data/test/behaviors.tsv"  
test_news_file = "/kaggle/input/mind-data/mind_data/test/news.tsv"

print("🔧 Patching iterator for test set (no labels)...")

# Backup original method
original_init = model.test_iterator.init_behaviors

def init_behaviors_no_labels(behaviors_file):
    """Modified init_behaviors for test set without labels"""
    model.test_iterator.histories = []
    model.test_iterator.imprs = []
    model.test_iterator.labels = []
    model.test_iterator.impr_indexes = []
    model.test_iterator.uindexes = []

    with open(behaviors_file, "r", encoding="utf-8") as rd:
        impr_index = 0
        for line in rd:
            uid, time, history, impr = line.strip("\n").split("\t")[-4:]

            # Parse history
            history = [model.test_iterator.nid2index[i] for i in history.split() if i in model.test_iterator.nid2index]
            history = [0] * (model.test_iterator.his_size - len(history)) + history[:model.test_iterator.his_size]

            # Parse impressions - TEST SET KHÔNG CÓ LABEL
            impr_news = []
            for item in impr.split():
                # Test set: chỉ có news_id, KHÔNG có "-0" hay "-1"
                if "-" in item:
                    # Validation/train set format: N12345-1
                    news_id = item.split("-")[0]
                else:
                    # Test set format: N12345
                    news_id = item
                
                if news_id in model.test_iterator.nid2index:
                    impr_news.append(model.test_iterator.nid2index[news_id])
            
            # Tạo dummy labels (không dùng cho test)
            label = [0] * len(impr_news)
            
            uindex = model.test_iterator.uid2index[uid] if uid in model.test_iterator.uid2index else 0

            model.test_iterator.histories.append(history)
            model.test_iterator.imprs.append(impr_news)
            model.test_iterator.labels.append(label)
            model.test_iterator.impr_indexes.append(impr_index)
            model.test_iterator.uindexes.append(uindex)
            impr_index += 1

# Apply patch
model.test_iterator.init_behaviors = init_behaviors_no_labels

print("✅ Iterator patched for test set")

# Run evaluationa
print("\n🔍 Running evaluation on test set...")
print("   Model will use trained weights to generate rankings")

group_impr_indexes, group_labels, group_preds = model.run_fast_eval(
    test_news_file, test_behaviors_file
)

print(f"\n✅ Generated predictions for {len(group_impr_indexes)} impressions")

# Write predictions
print("\n💾 Writing predictions to file...")
prediction_file = "/kaggle/working/prediction.txt"

with open(prediction_file, 'w') as f:  
    for impr_index, preds in tqdm(zip(group_impr_indexes, group_preds), 
                                   total=len(group_impr_indexes),
                                   desc="Writing"):  
        # MIND competition format: impression_id bắt đầu từ 1
        impr_id = impr_index + 1
        
        # Calculate rankings (score cao nhất = rank 1)
        pred_rank = (np.argsort(np.argsort(preds)[::-1]) + 1).tolist()  
        pred_rank_str = '[' + ','.join([str(i) for i in pred_rank]) + ']'  
        
        f.write(f"{impr_id} {pred_rank_str}\n")

print(f"\n✅ Prediction file saved to: {prediction_file}")

# Validate output
print("\n🔍 Validating output format...")
with open(prediction_file, 'r') as f:
    lines = f.readlines()
    print(f"   Total predictions: {len(lines)}")
    print(f"   First 3 lines:")
    for i in range(min(3, len(lines))):
        parts = lines[i].strip().split()
        print(f"      ImprID={parts[0]}, Rankings={parts[1][:50]}{'...' if len(parts[1]) > 50 else ''}")

print("\n✅ File ready to submit to MIND competition!")
print(f"   Download: {prediction_file}")